# LRP baseline model


In [8]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.utils import class_weight

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers, regularizers

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pickle
import pathlib
import hashlib
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Dotenv
from dotenv import dotenv_values

# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_resnet import *
from utils.utils_plot import *
from utils.DNN_models import *

Num GPUs Available:  1


In [9]:
import yaml
conf = yaml.safe_load(open("config.yaml"))
PRECIP_XTRM = 0.95 # Percentile (threshold) for the extremes
PRECIP_DATA = 'ERA5-low' 

In [10]:
i_shape = conf['i_shape']
o_shape = conf['o_shape']

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')
output_channels = conf['output_channels']
num_filters = conf['num_filters']
use_batchnorm = conf['use_batchnorm']
dropout = conf['dropout']
lr = conf['lr']

name_model = conf['model']
output_scaling = 1
output_crop = None


# load coordinates
lons_x = np.load('tmp/data/lons_y.npy')
lats_y = np.load('tmp/data/lats_y.npy')

# load precip
if PRECIP_XTRM == 0.95:
    pr_xtrm = np.load('tmp/data/pr_xtrm_95.npy')
elif PRECIP_XTRM == 0.99:
    pr_xtrm = np.load('tmp/data/pr_xtrm_99.npy')
#pr_xtrm = np.load('tmp/data/pr_xtrm_99.npy')
# create a time array
times = np.arange(np.datetime64('1979-01-01'), np.datetime64('2006-01-01')) #until validation period
times = pd.to_datetime(times)

X shape: [46, 56, 31]
y shape: [46, 56, 1]


In [11]:
# load the training and testing data
dg_train_X = np.array(xr.open_dataarray('tmp/data/dg_train_X.nc'))
dg_train_Y = np.array(xr.open_dataarray('tmp/data/dg_train_Y.nc'))

dg_valid_X = np.array(xr.open_dataarray('tmp/data/dg_valid_X.nc'))
dg_valid_Y = np.array(xr.open_dataarray('tmp/data/dg_valid_Y.nc'))


test_times = np.arange(np.datetime64('2016-01-01'), np.datetime64('2022-01-01')) #until validation period
test_times = pd.to_datetime(test_times)
dg_test_X = np.array(xr.open_dataarray('tmp/data/dg_test_X.nc'))
dg_test_Y = np.array(xr.open_dataarray('tmp/data/dg_test_Y.nc'))

# Open files for extremes
if PRECIP_XTRM == 0.95:
    dg_train_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_train_Y_xtrm0.95th.nc'))
    dg_valid_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_valid_Y_xtrm0.95th.nc')) 
    dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm0.95th.nc'))
elif PRECIP_XTRM == 0.99:
    dg_train_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_train_Y_xtrm0.99th.nc'))
    dg_valid_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_valid_Y_xtrm0.99th.nc')) 
    dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm0.99th.nc'))

In [12]:
# We need to pass zeros through the network to asses the baseline
dg_train_X_null = np.zeros(dg_train_X.shape)
dg_valid_X_null = np.zeros(dg_valid_X.shape)

In [15]:
# Define hyperparameters
BATCH_SIZE = 64
LR_METHOD = 'Constant'  # Cyclical, CosineDecay, Constant
                                            


# Compute weights for the weighted binary crossentropy
weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(pr_xtrm),
    y=pr_xtrm.flatten()
)

print('Weights for the weighted binary crossentropy:')
print(f'Classes: {np.unique(pr_xtrm)}, weights: {weights}')

# Create loss function for the extremes
xtrm_loss = weighted_binary_cross_entropy(
    weights={0: weights[0].astype('float32'), 1: weights[1].astype('float32')})



# Define hyperparameters
EPOCHS = 10
LR_METHOD = 'Constant'  # Cyclical, CosineDecay, Constant
    
# Early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                            restore_best_weights=True)
                                            
# Default model options
opt_model = {'latent_dim': 128,
             'dropout_rate': 0.2}

# Default training options
opt_training = {'epochs': EPOCHS,
                'callbacks': [callback]}

# Default optimizer options
opt_optimizer = {'lr_method': 'Constant',
                 'lr': 0.0004,
                 'init_lr': 0.01}

models_unets = {
          'UNET1': {'model': 'Unet', 'run': False,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 1, 'use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET2': {'model': 'Unet', 'run': False,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 2, 'use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET3': {'model': 'Unet', 'run': False,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 3, 'use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET4': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 4, 'use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}}
            }


Weights for the weighted binary crossentropy:
Classes: [0 1], weights: [0.52634048 9.99109415]


In [16]:
models = models_unets

train_for_prec = True
train_for_xtrm = False
history_log_level = 1


# define loss function
loss_regression = 'mse'

models_prec = []
models_xtrm = []

In [17]:
if train_for_prec:
        
    for m_id in models:
        # Clear session and set tf seed
        keras.backend.clear_session()
        tf.random.set_seed(42)
        
        if not models[m_id]['run']:
            continue

        # Extract model name and options
        model = models[m_id]['model']
        opt_model_i = models[m_id]['opt_model']
        opt_optimizer_i = models[m_id]['opt_optimizer']
        opt_model_new = opt_model.copy()
        opt_model_new.update(opt_model_i)
        opt_optimizer_new = opt_optimizer.copy()
        opt_optimizer_new.update(opt_optimizer_i)
      
        
        optimizer = initiate_optimizer(**opt_optimizer_new)


        # Create the model and compile
        # Update: to apply lrp the last activation function is recommended to be linear (see innvestigate)
        m = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=False, for_lrp = True, **opt_model_new)
        # Warning: When using regularizers, the loss function is the entire loss, ie (loss metrics) + (regularization term)!
        # But the loss displayed as part of the metrics, is only the loss metric. The regularization term is not added there. -> can be different!!
        loss_fct = 'mse'
        if loss_regression == 'mse_nans':
            loss_fct = MeanSquaredErrorNans()
        
        m.model.compile(
                loss=loss_fct, 
                metrics=[loss_fct], 
                optimizer=optimizer
            )
        print(f'Number of parameters: {m.model.count_params()}')

        # Train with the NULL datasets to assess the baseline
        hist = m.model.fit(dg_train_X_null, dg_train_Y, validation_data=(dg_valid_X_null, dg_valid_Y), **opt_training)
        
        # Saving the model
        print('Saving weights')
        m.model.save_weights(f'tmp/Keras_Baseline/null_input/{PRECIP_DATA}_{PRECIP_XTRM}_{m_id}.h5')
        
        
if train_for_xtrm:

    for m_id in models:
        # Clear session and set tf seed
        keras.backend.clear_session()
        tf.random.set_seed(42)

        if not models[m_id]['run']:
            continue
        
        # Extract model name and options
        model = models[m_id]['model']
        opt_model_i = models[m_id]['opt_model']
        opt_optimizer_i = models[m_id]['opt_optimizer']
        opt_model_new = opt_model.copy()
        opt_model_new.update(opt_model_i)
        opt_optimizer_new = opt_optimizer.copy()
        opt_optimizer_new.update(opt_optimizer_i)
        print(f'Running: {m_id} - {model} - {opt_model_i} - {opt_optimizer_i}')
        
        
        optimizer = initiate_optimizer(**opt_optimizer_new)
        
     
        # Create the model and compile
        m = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=True, for_lrp = True, **opt_model_new)
        m.model.compile(
                loss=xtrm_loss,
                optimizer=optimizer
        )
        print(f'Number of parameters: {m.model.count_params()}')

        # Train
        hist = m.model.fit(dg_train_X_null, dg_train_Y, validation_data=(dg_valid_X_null, dg_valid_Y), verbose=history_log_level, **opt_training)
            
           
        # Saving the model
        m.model.save_weights(f'tmp/Keras_Baseline/null_input/{PRECIP_DATA}_{PRECIP_XTRM}_{m_id}_xtrm.h5')
        

2023-02-06 13:40:06.689475: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 13:40:07.335570: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


Number of parameters: 31418369
Epoch 1/10


2023-02-06 13:40:13.849846: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


309/309 [==============================] - 31s 80ms/step - loss: 7.3760 - mse: 7.3760 - val_loss: 5.3387 - val_mse: 5.3387
Epoch 2/10
309/309 [==============================] - 21s 67ms/step - loss: 5.1582 - mse: 5.1582 - val_loss: 4.6120 - val_mse: 4.6120
Epoch 3/10
309/309 [==============================] - 21s 68ms/step - loss: 4.5270 - mse: 4.5270 - val_loss: 4.1004 - val_mse: 4.1004
Epoch 4/10
309/309 [==============================] - 21s 68ms/step - loss: 4.1863 - mse: 4.1863 - val_loss: 4.0943 - val_mse: 4.0943
Epoch 5/10
309/309 [==============================] - 21s 68ms/step - loss: 3.8839 - mse: 3.8839 - val_loss: 3.5628 - val_mse: 3.5628
Epoch 6/10
309/309 [==============================] - 21s 68ms/step - loss: 3.7351 - mse: 3.7351 - val_loss: 3.4349 - val_mse: 3.4349
Epoch 7/10
309/309 [==============================] - 21s 68ms/step - loss: 3.5751 - mse: 3.5751 - val_loss: 3.3599 - val_mse: 3.3599
Epoch 8/10
309/309 [==============================] - 21s 68ms/step - los

### Additional test 

In [12]:
##############LRP ###########
EPOCHS = 10
m_id = 'UNET4'
model = models[m_id]['model']
opt_model_i = models[m_id]['opt_model']
opt_optimizer_i = models[m_id]['opt_optimizer']

opt_model_new = opt_model.copy()
opt_model_new.update(opt_model_i)
opt_optimizer_new = opt_optimizer.copy()
opt_optimizer_new.update(opt_optimizer_i)

In [13]:
# Extremes
m = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=True,**opt_model_new)

# compile 
m.model.compile(loss=keras.losses.categorical_crossentropy, ## instead of CategoricalCrossentropy
                  optimizer='adam', ## lr instead of learning_rate
                  metrics=['categorical_accuracy'])

In [14]:
# Train 
m.model.fit(dg_train_X, dg_train_Y_xtrm, validation_data=(dg_valid_X, dg_valid_Y_xtrm), 
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=0) 


2023-02-03 09:26:04.838321: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


In [15]:
m.model.save_weights('tmp/tmp_weights_DNN/UNET4_0.95th_xtrm_trained_weights.h5')

In [11]:
# Not extremes
m_pr = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=False,**opt_model_new)

# compile 
m_pr.model.compile( loss='mse',  metrics=['mse'],  optimizer='adam')

2023-02-03 09:34:43.495378: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 09:34:44.155315: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


In [12]:
m_pr.model.fit(dg_train_X, dg_train_Y, validation_data=(dg_valid_X, dg_valid_Y), 
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

Epoch 1/10


2023-02-03 09:35:06.262970: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


155/155 [==============================] - 29s 140ms/step - loss: 6.9837 - mse: 6.9837 - val_loss: 5.7747 - val_mse: 5.7747
Epoch 2/10
155/155 [==============================] - 18s 113ms/step - loss: 4.8747 - mse: 4.8747 - val_loss: 4.5110 - val_mse: 4.5110
Epoch 3/10
155/155 [==============================] - 18s 114ms/step - loss: 4.2975 - mse: 4.2975 - val_loss: 4.0320 - val_mse: 4.0320
Epoch 4/10
155/155 [==============================] - 18s 114ms/step - loss: 3.9479 - mse: 3.9479 - val_loss: 4.2180 - val_mse: 4.2180
Epoch 5/10
155/155 [==============================] - 18s 115ms/step - loss: 3.7540 - mse: 3.7540 - val_loss: 3.7675 - val_mse: 3.7675
Epoch 6/10
155/155 [==============================] - 18s 115ms/step - loss: 3.5726 - mse: 3.5726 - val_loss: 3.4176 - val_mse: 3.4176
Epoch 7/10
155/155 [==============================] - 18s 114ms/step - loss: 3.4416 - mse: 3.4416 - val_loss: 4.0747 - val_mse: 4.0747
Epoch 8/10
155/155 [==============================] - 18s 115ms/st

In [13]:
m_pr.model.save_weights('tmp/tmp_weights_DNN/UNET4_pr_trained_weights.h5')

In [13]:
# train the model for the baseline- pasing zero through the network, using linear activation function!

In [10]:
m_extr_null = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=True, for_lrp=True, **opt_model_new)

# compile 
m_extr_null.model.compile(loss=keras.losses.categorical_crossentropy, ## instead of CategoricalCrossentropy
                  optimizer='adam', ## lr instead of learning_rate
                  metrics=['categorical_accuracy'])

m_extr_null.model.fit(dg_train_X_null, dg_train_Y_xtrm, validation_data=(dg_valid_X_null, dg_valid_Y_xtrm), 
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

2023-02-03 09:41:03.977862: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 09:41:04.641172: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


Epoch 1/10


2023-02-03 09:41:12.242114: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


155/155 [==============================] - 29s 142ms/step - loss: nan - categorical_accuracy: 1.0000 - val_loss: nan - val_categorical_accuracy: 1.0000
Epoch 2/10
155/155 [==============================] - 17s 112ms/step - loss: nan - categorical_accuracy: 1.0000 - val_loss: nan - val_categorical_accuracy: 1.0000
Epoch 3/10
155/155 [==============================] - 17s 112ms/step - loss: nan - categorical_accuracy: 1.0000 - val_loss: nan - val_categorical_accuracy: 1.0000
Epoch 4/10
155/155 [==============================] - 17s 113ms/step - loss: nan - categorical_accuracy: 1.0000 - val_loss: nan - val_categorical_accuracy: 1.0000
Epoch 5/10
155/155 [==============================] - 17s 113ms/step - loss: nan - categorical_accuracy: 1.0000 - val_loss: nan - val_categorical_accuracy: 1.0000
Epoch 6/10
155/155 [==============================] - 17s 113ms/step - loss: nan - categorical_accuracy: 1.0000 - val_loss: nan - val_categorical_accuracy: 1.0000
Epoch 7/10
155/155 [=============

In [11]:
m_extr_null.model.save_weights('tmp/tmp_weights_DNN/UNET4_prxtrm_NULL_linear_trained_weights.h5')

In [9]:
# Baseline non extremes

m_null = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=False, for_lrp=True, **opt_model_new)

# compile 
m_null.model.compile( loss='mse',  metrics=['mse'],  optimizer='adam')

m_null.model.fit(dg_train_X_null, dg_train_Y, validation_data=(dg_valid_X_null, dg_valid_Y), 
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

2023-02-03 09:48:14.644171: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 09:48:15.308870: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


Epoch 1/10


2023-02-03 09:48:22.814061: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


155/155 [==============================] - 29s 141ms/step - loss: 22.1827 - mse: 22.1827 - val_loss: 23.0957 - val_mse: 23.0957
Epoch 2/10
155/155 [==============================] - 17s 112ms/step - loss: 21.5591 - mse: 21.5590 - val_loss: 22.4856 - val_mse: 22.4856
Epoch 3/10
155/155 [==============================] - 17s 112ms/step - loss: 20.9999 - mse: 20.9999 - val_loss: 21.9388 - val_mse: 21.9388
Epoch 4/10
155/155 [==============================] - 17s 112ms/step - loss: 20.5009 - mse: 20.5009 - val_loss: 21.4504 - val_mse: 21.4504
Epoch 5/10
155/155 [==============================] - 17s 112ms/step - loss: 20.0577 - mse: 20.0577 - val_loss: 21.0158 - val_mse: 21.0158
Epoch 6/10
155/155 [==============================] - 17s 113ms/step - loss: 19.6651 - mse: 19.6651 - val_loss: 20.6308 - val_mse: 20.6308
Epoch 7/10
155/155 [==============================] - 17s 113ms/step - loss: 19.3203 - mse: 19.3203 - val_loss: 20.2928 - val_mse: 20.2928
Epoch 8/10
155/155 [==================

In [10]:
m_null.model.save_weights('tmp/tmp_weights_DNN/UNET4_pr_NULL_linear_trained_weights.h5')

#### Assess relevances (check the LRP_visualization notebook that uses other kernel)
### End